In [1]:
import numpy as onp 
import data_prep

NPOINT = 1024
NMASK = 10
TRAIN_DATASET = data_prep.FlowDataset('data/flow_train.mat', npoint=NPOINT)

def get_batch_data(dataset, idxs, start_idx, end_idx):
    bsize = end_idx-start_idx
    batch_pcpair = onp.zeros((bsize, NPOINT, 6), dtype=onp.float)
    batch_flow = onp.zeros((bsize, NPOINT, 3), dtype=onp.float)
    batch_vismask = onp.zeros((bsize, NPOINT), dtype=onp.float)
    batch_momasks = onp.zeros((bsize, NMASK, NPOINT), dtype=onp.float)
    for i in range(bsize):
        pc1, pc2, flow12, vismask, momasks = dataset[idxs[i+start_idx]]
        batch_pcpair[i,...] = onp.concatenate((pc1,pc2), 1)
        batch_flow[i,...] = flow12
        batch_vismask[i,:] = vismask
        batch_momasks[i,...] = onp.transpose(momasks)
    return batch_pcpair, batch_flow, batch_vismask, batch_momasks

train_idxs = onp.arange(0, len(TRAIN_DATASET))
start_idx = 0
end_idx = len(TRAIN_DATASET)
batch_pcpair, batch_flow, batch_vismask, batch_momasks = get_batch_data(TRAIN_DATASET, train_idxs, start_idx, end_idx)

In [2]:
xyz1s, xyz2s = onp.split(batch_pcpair, indices_or_sections=2, axis=2)

In [3]:
idx = 200

xyz1 = xyz1s[idx]
xyz2 = xyz2s[idx]

In [4]:
def procrustes(X, Y, scaling=True, reflection='best'):
    """
    Taken from
    https://stackoverflow.com/questions/18925181/procrustes-analysis-with-numpy
    
    A port of MATLAB's `procrustes` function to Numpy.

    Procrustes analysis determines a linear transformation (translation,
    reflection, orthogonal rotation and scaling) of the points in Y to best
    conform them to the points in matrix X, using the sum of squared errors
    as the goodness of fit criterion.

        d, Z, [tform] = procrustes(X, Y)

    Inputs:
    ------------
    X, Y    
        matrices of target and input coordinates. they must have equal
        numbers of  points (rows), but Y may have fewer dimensions
        (columns) than X.

    scaling 
        if False, the scaling component of the transformation is forced
        to 1

    reflection
        if 'best' (default), the transformation solution may or may not
        include a reflection component, depending on which fits the data
        best. setting reflection to True or False forces a solution with
        reflection or no reflection respectively.

    Outputs
    ------------
    d       
        the residual sum of squared errors, normalized according to a
        measure of the scale of X, ((X - X.mean(0))**2).sum()

    Z
        the matrix of transformed Y-values

    tform   
        a dict specifying the rotation, translation and scaling that
        maps X --> Y

    """
    
    np = onp

    n,m = X.shape
    ny,my = Y.shape

    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY

    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    if reflection is not 'best':

        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0

        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)

    traceTA = s.sum()

    if scaling:

        # optimum scaling of Y
        b = traceTA * normX / normY

        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2

        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX

    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX

    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)

    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}

    return d, Z, tform


In [60]:
# Motion segmentation
import numpy.random as onpr

onpr.seed(0)

max_iter = 10
num_point = xyz1.shape[0]

lams = [None] * num_point
link = 0
it = 0

tau = 0.05
min_num_inliers = 100

while it < max_iter:
    
    i, j, k = onpr.choice(num_point, size=3, replace=False)
    
    d, Z, tform = procrustes(
        onp.array([xyz1[i], xyz1[j], xyz1[k]]),
        onp.array([xyz2[i], xyz2[j], xyz2[k]]),
    )

    # I think these maps xyz2 to xyz1
    R, t, sigma = tform['rotation'], tform['translation'], tform['scale']
    
    num_inliers = 0
    
    for i in range(num_point):
        # formula from https://www.mathworks.com/help/stats/procrustes.html
        xyz2_pp = sigma * onp.dot(xyz2[i], R) + t
        
        if onp.linalg.norm(xyz1[i] - xyz2_pp) < tau and lams[i] is None:
            lams[i] = 'temp'
            num_inliers += 1
            
#         if i > 0 and i % 500 == 0:
#             print('finished', i)
                
    print(f'num inliers: {num_inliers}')
        
    if num_inliers > min_num_inliers:
        new_label = link
        link += 1
        it = 0

    else:
        new_label = None 
        it += 1
    
    for i in range(num_point):
        if lams[i] == 'temp':
            lams[i] = new_label
            
    num_unmatched = sum([1 for i in range(num_point) if lams[i] is None])
    print(f'num_point_unmatched: {num_unmatched}')
    print()

num inliers: 395
num_point_unmatched: 629

num inliers: 42
num_point_unmatched: 629

num inliers: 63
num_point_unmatched: 629

num inliers: 13
num_point_unmatched: 629

num inliers: 50
num_point_unmatched: 629

num inliers: 285
num_point_unmatched: 344

num inliers: 54
num_point_unmatched: 344

num inliers: 33
num_point_unmatched: 344

num inliers: 156
num_point_unmatched: 188

num inliers: 0
num_point_unmatched: 188

num inliers: 0
num_point_unmatched: 188

num inliers: 11
num_point_unmatched: 188

num inliers: 24
num_point_unmatched: 188

num inliers: 4
num_point_unmatched: 188

num inliers: 0
num_point_unmatched: 188

num inliers: 0
num_point_unmatched: 188

num inliers: 0
num_point_unmatched: 188

num inliers: 136
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inliers: 0
num_point_unmatched: 52

num inl

In [61]:
num_none = 0
num_0 = 0
num_1 = 0

print(len(lams))
for i in range(num_point):
    
    if lams[i] is None: num_none += 1
    if lams[i] == 0: num_0 += 1
    if lams[i] == 1: num_1 += 1
        
print(num_none, num_0, num_1)

1024
52 395 285


In [62]:
xyz1_no_match = []
xyz2_no_match = []

num_match = onp.nanmax(onp.array(lams, dtype=onp.float32)) + 1
num_match = int(num_match)

xyz1_match = [[] for i in range(num_match)]
xyz2_match = [[] for i in range(num_match)]

for i in range(num_point):
    
    if lams[i] is None:
        xyz1_no_match.append(xyz1[i])
        xyz2_no_match.append(xyz2[i])
        continue
    
    part_idx = int(lams[i])
    
    xyz1_match[part_idx].append(xyz1[i])
    xyz2_match[part_idx].append(xyz2[i])

In [63]:
import open3d as o3d

viz_offset = 0.5
pcds = []

for i, (xyz1_pts, xyz2_pts) in enumerate(zip([xyz1_no_match] + xyz1_match, [xyz2_no_match] + xyz2_match)):
    print(len(xyz1_pts), len(xyz2_pts))
    
    if i == 0:
        color = [1.0, 0.0, 0.0]
    else:
        color = onpr.rand(3, )
    
    xyz1_pcd = o3d.geometry.PointCloud()
    xyz1_pcd.points = o3d.utility.Vector3dVector(xyz1_pts)
    xyz1_pcd.paint_uniform_color(color)
    
    xyz2_pcd = o3d.geometry.PointCloud()
    xyz2_pcd.points = o3d.utility.Vector3dVector(onp.array(xyz2_pts) + viz_offset)
    xyz2_pcd.paint_uniform_color(color)
    
    pcds.extend([xyz1_pcd, xyz2_pcd])
    
print(len(pcds))

52 52
395 395
285 285
156 156
136 136
10


In [64]:
o3d.visualization.draw_geometries(pcds)